<a href="https://colab.research.google.com/github/AkshayaKondaparthy/fake-news-detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1.Setup – Install & Import Libraries

In [ ]:
!pip install nltk


##2.Load Fake News Dataset from GitHub

In [ ]:

train_url = "https://github.com/PhilChodrow/PIC16b/raw/master/datasets/fake_news_train.csv"


In [ ]:
import pandas as pd

train_url = "https://github.com/PhilChodrow/PIC16b/raw/master/datasets/fake_news_train.csv"
data = pd.read_csv(train_url)
data.head()


,Unnamed: 0,title,text,fake
0,17366,Merkel: Strong result for Austria's FPO 'big c...,German Chancellor Angela Merkel said on Monday...,0
1,5634,Trump says Pence will lead voter fraud panel,"WEST PALM BEACH, Fla.President Donald Trump sa...",0
2,17487,JUST IN: SUSPECTED LEAKER and “Close Confidant...,"On December 5, 2017, Circa s Sara Carter warne...",1
3,12217,Thyssenkrupp has offered help to Argentina ove...,"Germany s Thyssenkrupp, has offered assistance...",0
4,5535,Trump say appeals court decision on travel ban...,President Donald Trump on Thursday called the ...,0


##3.Clean Text (title + text)

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Combine title and text
data['content'] = data['title'] + " " + data['text']

# Function to clean text
def clean_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)         # Remove punctuation
    text = text.lower()                            # Convert to lowercase
    words = text.split()                           # Tokenize
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply the cleaning function
data['clean_content'] = data['content'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##4.Convert Text to Vectors (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['clean_content']).toarray()
y = data['fake']  # Target variable: 0 = Real, 1 = Fake


##5.Split Data into Train and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


##6.Train a Classifier (Naive Bayes)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

## 7.Evaluate the Model

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


✅ Accuracy: 0.9285077951002227

📊 Confusion Matrix:
 [[1969  160]
 [ 161 2200]]

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92      2129
           1       0.93      0.93      0.93      2361

    accuracy                           0.93      4490
   macro avg       0.93      0.93      0.93      4490
weighted avg       0.93      0.93      0.93      4490



## 8.Predict on Custom Input

In [ ]:
def predict_news(text):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])
    prediction = model.predict(vector)[0]
    return "🧾 Real News" if prediction == 0 else "🚨 Fake News"

# Try it out
predict_news("Breaking: NASA Confirms Alien Life on Mars")


'🚨 Fake News'